Deliverables : 
    
    1. TextBox++ model
    2. Model can be run through the Utils provided
    3. A .h5 format file can also be used to run
    4. A protobuf file, to run the model onDevice
    5. An app that uses a Server to run the model
   

In [ ]:
import os
import time


In [ ]:

os.chdir("Desktop/SamsungResearch/submission")


In [ ]:
import tensorflow as tf
print(tf.__version__)


In [ ]:
import numpy as np
import cv2
import tensorflow as tf
from timeit import default_timer as timer

# textbox++ models
from tbpp_model import TBPP512, TBPP512_dense
from tbpp_utils import PriorUtil

# ssd for help
from ssd_data import preprocess
from sl_utils import rbox3_to_polygon, polygon_to_rbox, rbox_to_polygon

In [ ]:
if __name__ == '__main__':
    Model = TBPP512_dense
    input_shape = (512,512,3)
    weights_path = 'weights.022.h5'
    confidence_threshold = 0.35
    confidence_threshold = 0.25

    sl_graph = tf.Graph()
    with sl_graph.as_default():
        sl_session = tf.Session()
        with sl_session.as_default():
            sl_model = Model(input_shape)
            prior_util = PriorUtil(sl_model)
            sl_model.load_weights(weights_path, by_name=True)


    input_width = 256
    input_height = 32
    weights_path = 'weights.022.h5'


In [ ]:

start_time = time.time()

# Final TextBox++ Code : (Works on just image)
input_size = input_shape[:2]

# model to predict
img = cv2.imread('slant.jpeg')

print(img.shape)
img_h = img.shape[0]
img_w = img.shape[1]
print(img_h)
print(img_w)

img1 = np.copy(img)
img2 = np.zeros_like(img)

# model to predict
x = np.array([preprocess(img, input_size)])



elapsed_time = time.time() - start_time
print("Performace measure : "+str(elapsed_time))

#Model start
start_time = time.time()
with sl_graph.as_default():
    with sl_session.as_default():
        y = sl_model.predict(x)

elapsed_time = time.time() - start_time
print("Performace measure : "+str(elapsed_time))
#Model end


start_time = time.time()
result = prior_util.decode(y[0], confidence_threshold)

if len(result) > 0:
    bboxs = result[:,0:4]
    quads = result[:,4:12]
    rboxes = result[:,12:17]

    boxes = np.asarray([rbox3_to_polygon(r) for r in rboxes])

    xy = boxes
    xy = xy * [img_w, img_h]
    xy = np.round(xy)
    xy = xy.astype(np.int32)

    cv2.polylines(img1, tuple(xy), True, (0,0,255))

    rboxes = np.array([polygon_to_rbox(b) for b in np.reshape(boxes, (-1,4,2))])
    bh = rboxes[:,3]
    rboxes[:,2] += bh * 0.1
    rboxes[:,3] += bh * 0.2
    boxes = np.array([rbox_to_polygon(f) for f in rboxes])

    boxes = np.flip(boxes, axis=1) # TODO: fix order of points, why?
    boxes = np.reshape(boxes, (-1, 8))

    boxes_mask_a = np.array([b[2] > b[3] for b in rboxes]) # width > height, in square world
    boxes_mask_b = np.array([not (np.any(b < 0) or np.any(b > 512)) for b in boxes]) # box inside image
    boxes_mask = np.logical_and(boxes_mask_a, boxes_mask_b)

    boxes = boxes[boxes_mask]
    rboxes = rboxes[boxes_mask]
    xy = xy[boxes_mask]

    if len(boxes) == 0:
        boxes = np.empty((0,8))

# draw fps
cv2.rectangle(img1, (0,0), (50, 17), (255,255,255), -1)
cv2.imwrite('Testout.jpg', img1)
print("DONE!")
elapsed_time = time.time() - start_time
print("Performace measure : "+str(elapsed_time))


In [ ]:
# EXTRA IMPLEMENTATION
import keras
from keras import backend as K 
import numpy as np
import cv2
import tensorflow as tf
from timeit import default_timer as timer

from keras.models import load_model
from utils.layers import Normalize


In [ ]:
model_load_through_h5 = load_model('sl_my_model.h5',custom_objects={'Normalize': Normalize})
print(dir(model_load_through_h5))


In [ ]:
model_load_through_h5.load_weights(weights_path, by_name=True)


In [ ]:
def preprocess(img, size):
    """Precprocess an image for ImageNet models.
    
    # Arguments
        img: Input Image
        size: Target image size (height, width).
    
    # Return
        Resized and mean subtracted BGR image, if input was also BGR.
    """
    h, w = size
    img = np.copy(img)
    img = cv2.resize(img, (w,h), cv2.INTER_LINEAR)
    img = img.astype(np.float32)
    mean = np.array([104,117,123])
    img -= mean[np.newaxis, np.newaxis, :]
    return img



In [ ]:

start_time = time.time()

# Final TextBox++ Code : (Works on just image)
input_size = input_shape[:2]

# model to predict
img = cv2.imread('slant.jpeg')

print(img.shape)
img_h = img.shape[0]
img_w = img.shape[1]
print(img_h)
print(img_w)

img1 = np.copy(img)
img2 = np.zeros_like(img)

# model to predict
x = np.array([preprocess(img, input_size)])



elapsed_time = time.time() - start_time
print("Performace measure : "+str(elapsed_time))

#Model start
y = model_load_through_h5.predict(x)
elapsed_time = time.time() - start_time
print("Performace measure : "+str(elapsed_time))
#Model end


start_time = time.time()
result = prior_util.decode(y[0], confidence_threshold)

if len(result) > 0:
    bboxs = result[:,0:4]
    quads = result[:,4:12]
    rboxes = result[:,12:17]

    boxes = np.asarray([rbox3_to_polygon(r) for r in rboxes])

    xy = boxes
    xy = xy * [img_w, img_h]
    xy = np.round(xy)
    xy = xy.astype(np.int32)

    cv2.polylines(img1, tuple(xy), True, (0,0,255))

    rboxes = np.array([polygon_to_rbox(b) for b in np.reshape(boxes, (-1,4,2))])
    bh = rboxes[:,3]
    rboxes[:,2] += bh * 0.1
    rboxes[:,3] += bh * 0.2
    boxes = np.array([rbox_to_polygon(f) for f in rboxes])

    boxes = np.flip(boxes, axis=1) # TODO: fix order of points, why?
    boxes = np.reshape(boxes, (-1, 8))

    boxes_mask_a = np.array([b[2] > b[3] for b in rboxes]) # width > height, in square world
    boxes_mask_b = np.array([not (np.any(b < 0) or np.any(b > 512)) for b in boxes]) # box inside image
    boxes_mask = np.logical_and(boxes_mask_a, boxes_mask_b)

    boxes = boxes[boxes_mask]
    rboxes = rboxes[boxes_mask]
    xy = xy[boxes_mask]

    if len(boxes) == 0:
        boxes = np.empty((0,8))

# draw fps
cv2.rectangle(img1, (0,0), (50, 17), (255,255,255), -1)
cv2.imwrite('Testout.jpg', img1)
print("DONE!")
elapsed_time = time.time() - start_time
print("Performace measure : "+str(elapsed_time))
